# Read this first 

* each comment and text line added here is important and relevant , dont skim through just the code
* if the comment asks you to revisit some concepts/modules , do so
* this is not complete code on its own , you need to fill in your own code to eventually start building a model

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd 
import numpy as np

In [2]:
datafile_train="/Users/lalitsachan/Dropbox/March onwards/PDS V2/Projects/P4/Hazard_train.csv"
datafile_test="/Users/lalitsachan/Dropbox/March onwards/PDS V2/Projects/P4/Hazard_test_share.csv"
bd_train=pd.read_csv(datafile_train)
bd_test=pd.read_csv(datafile_test)

In [3]:
bd_train.head()

,Id,Hazard,T1_V1,T1_V2,T1_V3,T1_V4,T1_V5,T1_V6,T1_V7,T1_V8,...,T2_V6,T2_V7,T2_V8,T2_V9,T2_V10,T2_V11,T2_V12,T2_V13,T2_V14,T2_V15
0,83642,2,10,23,4,N,K,Y,B,D,...,2,37,1,11,2,Y,N,E,2,1
1,92530,6,3,18,2,B,H,N,B,B,...,2,34,1,12,7,Y,N,C,3,3
2,86131,4,5,20,4,B,K,N,B,B,...,1,40,1,17,4,Y,N,D,1,1
3,88666,1,4,7,5,N,H,N,B,B,...,2,37,1,11,6,Y,N,D,2,1
4,11122,21,10,21,6,N,C,Y,B,C,...,2,40,1,6,6,N,N,D,5,5


In [4]:
cat_cols=bd_train.select_dtypes(['object']).columns

In [5]:
cat_cols

Index(['T1_V4', 'T1_V5', 'T1_V6', 'T1_V7', 'T1_V8', 'T1_V9', 'T1_V11',
       'T1_V12', 'T1_V15', 'T1_V16', 'T1_V17', 'T2_V3', 'T2_V5', 'T2_V11',
       'T2_V12', 'T2_V13'],
      dtype='object')

In [6]:
bd_train['data']='train'
bd_test['data']='test'
all_data=pd.concat([bd_train,bd_test],0,sort=False)

In [7]:
all_data.shape

(50999, 35)

In [8]:
for col in cat_cols:
    k=all_data[col].value_counts()
    cats=k[k>=100].index[:-1]
    
    for cat in cats:
        name=col+'_'+cat
        all_data[name]=(all_data[col]==cat).astype(int)
        
    del all_data[col]

In [9]:
all_data.shape

(50999, 94)

In [10]:
all_data.head()

,Id,Hazard,T1_V1,T1_V2,T1_V3,T1_V10,T1_V13,T1_V14,T2_V1,T2_V2,...,T2_V5_A,T2_V5_B,T2_V5_C,T2_V5_D,T2_V11_Y,T2_V12_N,T2_V13_E,T2_V13_A,T2_V13_C,T2_V13_D
0,83642,2.0,10,23,4,2,10,0,52,14,...,0,0,1,0,1,1,1,0,0,0
1,92530,6.0,3,18,2,8,10,3,39,14,...,1,0,0,0,1,1,0,0,1,0
2,86131,4.0,5,20,4,7,15,1,66,10,...,0,0,1,0,1,1,0,0,0,1
3,88666,1.0,4,7,5,8,15,3,48,11,...,0,1,0,0,1,1,0,0,0,1
4,11122,21.0,10,21,6,12,10,1,42,11,...,1,0,0,0,0,1,0,0,0,1


In [11]:
x_train=all_data.drop(['Id','Hazard','data'],1)[all_data['data']=='train']
y_train=all_data['Hazard'][all_data['data']=='train']

In [12]:
x_train.shape, y_train.shape

((40799, 91), (40799,))

In [13]:
x_test=all_data.drop(['Id','Hazard','data'],1)[all_data['data']=='test']

In [14]:
from sklearn.linear_model import Lasso

# you should try other algos for improvement in performance 
# the target here is not really suitable for linear regression or regression cost in general
# as it takes always positive integer values 
# you should try PoissonRegressor available with sklearn .23 and onwards 
# if that is not available , you can also make use of objective :'count:poisson' with xgboost
# you can also try HistGradientBoostingRegressor with loss='poisson'


In [15]:
params={'alpha':np.linspace(0.01,10,100)}

In [16]:
from sklearn.model_selection import GridSearchCV

In [17]:
model=Lasso(fit_intercept=True)

In [18]:
grid_search=GridSearchCV(model,cv=10,scoring='neg_mean_absolute_error',
                         param_grid=params,n_jobs=-1,verbose=10)

# you can also make use of mean_poisson_deviance as scoring , this ignores non-positive outcomes
# it might not be avalaible with your version

In [19]:
grid_search.fit(x_train,y_train)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done  61 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done  74 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done  89 tasks      | elapsed:    8.3s
[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed:    8.9s
[Parallel(n_jobs=-1)]: Done 121 tasks      | elapsed:    9.7s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   10.4s
[Parallel(n_jobs=-1)]: Done 157 tasks      | elapsed:   11.2s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   11.9s
[Parallel(n_jobs=-1)]: Done 197 tasks      | elapsed:  

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'alpha': array([ 0.01   ,  0.11091, ...,  9.89909, 10.     ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_absolute_error', verbose=10)

In [20]:
grid_search.best_estimator_

Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [21]:
# you can see that best estimator is at the edge of the alpha range that we tried . You
# can expand on that, if you are using Lasso

In [27]:
abs(grid_search.best_score_)

2.748947514698889

In [22]:
submissions=pd.DataFrame({'Hazard':grid_search.predict(x_test)})

In [23]:
submissions.to_csv('sample_submission.csv',index=False)